In [1]:
from pathlib import Path
from itertools import chain
import pandas as pd
import requests
import numpy as np
import networkx as nx
from krippendorff_graph import (compute_alpha, graph_edit_distance, graph_overlap_metric,
                                nominal_metric, node_overlap_metric, compute_distance_matrix)
from ast import literal_eval


In [7]:
#save_path = f"./export/{metric_type}_distance_matrix_{feature_column}_{'_'.join([str(annotator) for annotator in annotator_list])}.npy"

#if not forced and Path(save_path).exists():
#    distance_matrix = np.load(save_path)
#else:
df = pd.read_csv("../../../data/annotated/pilot_annotation.csv")
print(df["annotator"].unique())
df['triples'] = df.triples.apply(literal_eval)
df = df.mask(df.map(str).eq('set()'))
df = df.replace({np.nan: "*"})

def clean_triples(lst):
    if lst == "*":
        return "*"
    else:
        # Filter out tuples with None in the first position
        filtered = {t for t in lst if t[0] is not None}
        filtered = {t for t in filtered if t[-1] is not None}
        # Return '*' if empty, otherwise return the list
        return filtered if filtered else "*"
df['triples'] = df['triples'].apply(clean_triples)
#df.replace({"pilot_3_min": "pilot_3", "pilot_3_max": "pilot_3"}, inplace=True)

pilot_id_list = df["pilot"].unique().tolist()
print(pilot_id_list)
for pilot_id in pilot_id_list:
    print("pilot_id:", pilot_id)
    df_sub=df[df["pilot"] == pilot_id]
    print(df_sub["annotator"].unique().tolist())
    data = [df_sub[df_sub["annotator"] == annotator_id]["triples"].to_list() for annotator_id in df_sub["annotator"].unique().tolist()]
    distance_matrix = compute_distance_matrix(df_sub, feature_column="triples",
                                                graph_distance_metric=graph_edit_distance, #graph_overlap_metric, graph_edit_distance
                                                empty_graph_indicator="*", save_path="./graph_edit_distance_matrix.npy", #"./graph_edit_distance_matrix.npy", graph_overlap_distance_matrix 
                                                graph_type=nx.DiGraph, timeout=60)

    alpha = compute_alpha(data, distance_matrix=distance_matrix, missing_items="*")
    print(f" {pilot_id} alpha: {alpha:.4f}")

['aaron' 'ida2024' 'samuel']
['pilot_1', 'pilot_2', 'pilot_3']
pilot_id: pilot_1
['aaron', 'ida2024', 'samuel']


100%|██████████| 7875/7875 [00:00<00:00, 10763.83it/s]


 pilot_1 alpha: 0.0909
pilot_id: pilot_2
['aaron', 'ida2024', 'samuel']


100%|██████████| 5151/5151 [00:00<00:00, 45965.64it/s]


 pilot_2 alpha: -0.2000
pilot_id: pilot_3
['ida2024', 'samuel']


100%|██████████| 2850/2850 [00:00<00:00, 4637.24it/s]

 pilot_3 alpha: -0.0714


In [3]:
data[0][17] 

{('Demand (Residual)', 'Increase', 'Inflation')}

In [4]:
distance_matrix[17, 17+len(data[0])]

1.0

In [5]:
df_sub=df[df["pilot"].isin(["pilot_3_min", "pilot_3_max"])]#, "pilot_2"])]
print(df_sub["annotator"].unique().tolist())
data = [df_sub[df_sub["annotator"] == annotator_id]["triples"].to_list() for annotator_id in df_sub["annotator"].unique().tolist()]
distance_matrix = compute_distance_matrix(df_sub, feature_column="triples",
                                            graph_distance_metric=graph_edit_distance,
                                            empty_graph_indicator="*", save_path="./graph_edit_distance_matrix.npy",
                                            graph_type=nx.DiGraph, timeout=60)

alpha = compute_alpha(data, distance_matrix=distance_matrix, missing_items="*")
print(f" {pilot_id} alpha: {alpha:.4f}")

[]


0it [00:00, ?it/s]


IndexError: list index out of range

In [ ]:

non_zero_indices = np.nonzero(distance_matrix)
print(non_zero_indices)

(array([ 0,  0,  0, ..., 70, 71, 71]), array([ 1,  2,  3, ..., 63, 27, 63]))


In [ ]:
for pilot_id in pilot_id_list:
    print("pilot_id:", pilot_id)
    df_sub=df[df["pilot"] == pilot_id]
    for doc_id in df_sub["doc_id"].unique().tolist():
        print("doc_id:", doc_id)
        annotators = df_sub[df_sub["doc_id"] == doc_id]["annotator"].unique().tolist()
        for annotator in annotators:
            print("annotator:", annotator)
            triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
            print("triples:", len(triples))
            print("triples:", triples.tolist())
        

/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]


pilot_id: pilot_1
doc_id: narrative_20
annotator: aaron
triples: 4
triples: ['*', '*', '*', '*']
annotator: ida2024
triples: 4
triples: ['*', '*', '*', '*']
annotator: samuel
triples: 4
triples: ['*', '*', '*', '*']
doc_id: narrative_19
annotator: aaron
triples: 4
triples: ['*', '*', '*', '*']
annotator: ida2024
triples: 4
triples: ['*', '*', '*', '*']
annotator: samuel
triples: 4
triples: ['*', '*', '*', '*']
doc_id: narrative_18
annotator: aaron
triples: 4
triples: ['*', '*', '*', '*']
annotator: ida2024
triples: 4
triples: ['*', '*', '*', '*']
annotator: samuel
triples: 4
triples: ['*', '*', '*', '*']
doc_id: narrative_16
annotator: aaron
triples: 4
triples: ['*', '*', '*', '*']
annotator: ida2024
triples: 4
triples: ['*', '*', '*', '*']
annotator: samuel
triples: 4
triples: ['*', '*', '*', '*']
doc_id: narrative_17
annotator: aaron
triples: 4
triples: ['*', '*', '*', '*']
annotator: ida2024
triples: 4
triples: [{('Demand (Residual)', 'Increase', 'Inflation')}, {('Demand (Residual)'

/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]
/var/folders/cm/3bzwnl_s7n90ryczc4tgbl_r0000gn/T/ipykernel_26229/2578032160.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  triples = df_sub[df_sub["doc_id"] == doc_id][df_sub["annotator"] == annotator]["triples"]


In [ ]:
df

,doc_id,text,annotator,pilot,doc_label,triples,spans,relations
0,narrative_20,USDA National Pork Midday FOB Plant Report - O...,aaron,pilot_1,non-inflation-related,*,[],[]
1,narrative_20,USDA National Pork Midday FOB Plant Report - O...,ida2024,pilot_1,non-inflation-related,*,[],[]
2,narrative_20,USDA National Pork Midday FOB Plant Report - O...,samuel,pilot_1,non-inflation-related,*,[],[]
3,narrative_19,Higher Rates Will Put the Kibosh on Consumptio...,aaron,pilot_1,inflation-related,*,[],[]
4,narrative_19,Higher Rates Will Put the Kibosh on Consumptio...,ida2024,pilot_1,inflation-related,*,[],[]
...,...,...,...,...,...,...,...,...
428,max_0,RBA Remains Dovish Even as GDP Forecasts Revis...,samuel,pilot_3_max,inflation-cause-dominant,"{(Labor Shortage, Increase, Inflation)}","[custom_Span(xmiID=6884, Is_Nuclei='Constituen...","[custom_Relation(xmiID=6891, Dependent=custom_..."
429,max_1,News Highlights: Top Global Markets News of th...,ida2024,pilot_3_max,inflation-related,*,[],[]
430,max_1,News Highlights: Top Global Markets News of th...,samuel,pilot_3_max,inflation-cause-dominant,"{(Russia-Ukraine War, Increase, Inflation)}","[custom_Span(xmiID=5114, Is_Nuclei='Constituen...","[custom_Relation(xmiID=5128, Dependent=custom_..."
431,max_10,Malaysia Central Bank Holds Policy Rate at 3.0...,ida2024,pilot_3_max,inflation-cause-dominant,*,"[custom_Span(xmiID=2775, Is_Nuclei='Constituen...","[custom_Relation(xmiID=2789, Dependent=custom_..."
